[![Colab-en ireki](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uxue-sudupe/API-Eustat/blob/main/code_examples/eu/tutorial_Python_eu.ipynb)

# Nola erabili Eustaten APIa Pythonekin

[Eustaten datu bankuak](https://eu.eustat.eus/banku/id_All/indexArbol.html) Euskal Autonomia Erkidegoko gizarteari, ekonomiari eta biztanleriari buruzko datu ugari ditu.

Datuak publikoak dira eta modu desberdinetan eskura daitezke, besteak beste, [hemen dokumentatutako](https://github.com/uxue-sudupe/API-Eustat) APIaren bidez. APIak izugarri errazten du datuak eskuratzea eta berrerabiltzea. Hala ere, APIekin aurretik esperientziarik izan ezean, zaila izan daiteke ikertzaile, analista edo ikasle batentzat hura erabiltzea.

Tutorial honek gida azkar bat eskaintzen du Eustaten datuak APIaren bidez eskuratzeko. Datuak nola deskargatu, prozesatu eta Dataframe formatuko taulak nola sortu erakusten du.

Oso zabalduta dauden pakete batzuk erabili behar dira, *pandas* edo *requests* adibidez, eta *pyjstat* paketea, Pythonekin datuak *json-stat* formatuan tratatzeko espezifikoa dena.



In [ ]:
!pip install -q pyjstat
import json
import requests
import pandas as pd
from pyjstat import pyjstat

# 1. Eustaten datuen katalogoa (taulen zerrenda) eskuratu

Posible da Eustatek argitaratutako datu multzoen (taulen) zerrenda osoa ikustea eta jeistea, taula bakoitzaren identifikazio-kodea, izenburua eta azken eguneratze-data barne.

Eustaten datu bankuan dauden taulek osatzen dute katalogoa: https://eu.eustat.eus/banku/id_All/indexArbol.html

APIari egiten zaion eskaera batek hiru osagai nagusi izaten ditu: web helbide nagusia, funtziorako dei bat eta parametro multzo bat. Eustaten Datu Bankuko APIaren helbide nagusia honakoa da:

 `https://www.eustat.eus/bankupx/api/v1/[lang]/DB`

 Kasu honetan, APIari deia `GET` metodoaren bidez egiten zaio. Katalogoa lortzeko behar den parametro bakarra hizkuntza da (lang). Erantzuna JSON formatuan jasotzen da.

In [ ]:
# Eustaten datuen katalogoko sarbide puntua (endpoint)
url = "https://www.eustat.eus/bankupx/api/v1/eu/DB"

erantzuna = requests.get(url)
erantzuna_json = erantzuna.json()

print(json.dumps(erantzuna_json, indent=2, ensure_ascii=False)[:500])  # JSONaren lehen 500 karaktereen itxura ikusteko bakarrik, ez da beharrezko urratsa

[
  {
    "id": "PX__fe_inem06.px",
    "type": "t",
    "text": "Euskal AEko erregistratutako langabezia, lurralde eremuaren eta sexuaren arabera. 1997 - 2022",
    "updated": "2023-01-09T15:04:41"
  },
  {
    "id": "PX__feinem_inem06.px",
    "type": "t",
    "text": "Euskal AEko erregistratutako langabezia, lurralde eremuaren eta sexuaren arabera",
    "updated": "2021-02-17T10:04:22"
  },
  {
    "id": "PX__fepycl_pc02cm.px",
    "type": "t",
    "text": "Euskal AEko eta Estatuko kontsumora


In [ ]:
katalogoa = pd.DataFrame(erantzuna_json)    # erantzuna dataframe bihurtzen dugu honekin
katalogoa.head()

,id,type,text,updated
0,PX__fe_inem06.px,t,"Euskal AEko erregistratutako langabezia, lurra...",2023-01-09T15:04:41
1,PX__feinem_inem06.px,t,"Euskal AEko erregistratutako langabezia, lurra...",2021-02-17T10:04:22
2,PX__fepycl_pc02cm.px,t,Euskal AEko eta Estatuko kontsumorako prezioen...,2021-02-16T11:15:00
3,PX_0_fe_pc02cm.px,t,Euskal AEko eta Estatuko kontsumorako prezioen...,2022-02-15T10:31:14
4,PX_0_fe_pc02m.px,t,Euskal AEko eta Estatuko kontsumorako prezioen...,2022-02-15T10:31:14


## Hizkuntzak

Zerrenda bera gaztelaniaz edo ingelesez ere eska daiteke. Horretarako, URLko hizkuntza parametroa aldatzea besterik ez da behar:
`www.eustat.eus/bankupx/api/v1/[lang]/DB`

`[lang]` parametroa honako hiru aukeretako batekin ordezka daiteke: `eu` (euskera), `es` (gaztelania) edo `en` (ingelesa).

Adibidez, gaztelaniazko katalogoa eskuratzeko URL hau erabiliko litzateke:

In [ ]:
url = "https://www.eustat.eus/bankupx/api/v1/es/DB"
erantzuna = requests.get(url)
erantzuna_json = erantzuna.json()

catálogo = pd.DataFrame(erantzuna_json)     # erantzuna DataFrame bihurtzen dugu honekin
catálogo.head()

,id,type,text,updated
0,PX__fe_inem06.px,t,Paro registrado de la C.A. de Euskadi por ámbi...,2023-01-09T15:04:41
1,PX__feinem_inem06.px,t,Paro registrado de la C.A. de Euskadi por ámbi...,2021-02-17T10:04:22
2,PX__fepycl_pc02cm.px,t,Índice general de precios al consumo (IPC) por...,2021-02-16T11:15:00
3,PX_0_fe_pc02cm.px,t,Índice general de precios al consumo (IPC) por...,2022-02-15T10:31:14
4,PX_0_fe_pc02m.px,t,Índice general de precios al consumo (IPC) de ...,2022-02-15T10:31:14


# 2. Taula baten metadatuak eskuratu
Taula bateko metadatuek taularen izenburuari buruzko informazioa, taulan agertzen diren desagregazio-dimentsioak/aldagaiak eta hartzen dituzten balioei buruzko informazioa jasotzen dute. Metadatuetan kodeak eta testu deskribatzaileak datoz.

Taula jakin bateko metadatuetara sartzeko, interesatzen zaigun taularen "id" identifikazio kodea gehitu behar diogu datu bankuko url-ari. Helbideak egitura hau izango du:

 `www.eustat.eus/bankupx/api/v1/[lang]/DB/[id]`

Adibidez, taula hau interesatzen bazaigu: *Euskal AEko biztanleria zenbatetsia, lurralde historikoaren, sexuaren eta jaioturtearen arabera*, helbide honi egin behar diogu dei:

https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010154_cepv1_ep17.px


In [ ]:
url = "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010154_cepv1_ep17.px"
erantzuna = requests.get(url)
erantzuna_json = erantzuna.json()

print(json.dumps(erantzuna_json, indent=2, ensure_ascii=False)[:500])    # JSONaren lehen 500 karaktereen itxura ikusteko bakarrik, ez da beharrezko urratsa

{
  "title": "Euskal AEko biztanleria zenbatetsia, lurralde historikoaren, sexuaren eta jaioturtearen arabera. 1976/01/01 - 2024/01/01",
  "variables": [
    {
      "code": "territorio histórico",
      "text": "lurralde historikoa",
      "values": [
        "00",
        "01",
        "48",
        "20"
      ],
      "valueTexts": [
        "Euskal AE",
        "Araba/Álava",
        "Bizkaia",
        "Gipuzkoa"
      ]
    },
    {
      "code": "sexo",
      "text": "sexua",
      "values


In [ ]:
# "variables" izeneko zerrendaren edukia aterako dugu, aldagaien zerrenda, hori baita interesatzen zaiguna. Gero DataFrame bihurtuko dugu bere edukia aztertzeko

variables = erantzuna_json.get('variables', [])
aldagaiak = pd.DataFrame(variables)
aldagaiak

,code,text,values,valueTexts,time
0,territorio histórico,lurralde historikoa,"[00, 01, 48, 20]","[Euskal AE, Araba/Álava, Bizkaia, Gipuzkoa]",NaN
1,sexo,sexua,"[10, 20, 30]","[Guztira, Gizonak, Emakumeak]",NaN
2,año de nacimiento,jaiotze urtea,"[1000, 1109, 1110, 1120, 5800, 5810, 5820, 583...","[Guztira, 2023, 2022, 2021, 2020, 2019, 2018, ...",NaN
3,periodo,aldia,"[19760101, 19770101, 19780101, 19790101, 19800...","[1976/01/01, 1977/01/01, 1978/01/01, 1979/01/0...",True


Jaso dugun DataFrameak taulako aldagaiak biltzen ditu, baina haren balioak edo kategoriak ilara bakoitzaren barruan azaltzen dira, azpiegitura gisa.

Balioak kanpora ateratzen baditugu, aldagaien eta balioen arteko korrespondentzia erakusten duen taula bat sortu dezakegu, bai kodeak eta bai  etiketa deskribatzaileak dituena.


In [ ]:
# Zerrenda bat sortu dugu errenkadak gordetzeko
zerrenda = []

# DataFrameko aldagai bakoitza irakurri eta atributuak atera
for _, ilara in aldagaiak.iterrows():
    aldagaia = ilara['text']
    kategoria = ilara.get('values', [])
    etiketa = ilara.get('valueTexts', [])

    # Kode bakoitza bere etiketarekin lotu
    for kodea, etiketa in zip(kategoria, etiketa):
        zerrenda.append({
            'aldagaia': aldagaia,
            'kodea': kodea,
            'etiketa': etiketa
        })

# DataFrame bihurtu
balioen_taula = pd.DataFrame(zerrenda)
balioen_taula

,aldagaia,kodea,etiketa
0,lurralde historikoa,00,Euskal AE
1,lurralde historikoa,01,Araba/Álava
2,lurralde historikoa,48,Bizkaia
3,lurralde historikoa,20,Gipuzkoa
4,sexua,10,Guztira
...,...,...,...
203,aldia,20220101,2022/01/01
204,aldia,20220701,2022/07/01
205,aldia,20230101,2023/01/01
206,aldia,20230701,2023/07/01


# 3. Taula bateko datuak eskuratu

## Taula bateko datu guztiak eskuratzeko kontsulta

Tauletako datuak eskuratzeko, kontsulta bat egin behar dugu aurreko puntuak ikusi dugun helbide berean, `POST` funtzioarekin.

*pyjstat* izeneko modulua erabiliko dugu *json-stat* formatuan deskargatuko ditugun datuekin lan egiteko. Adibide honetan, *Euskal AEko barne produktu gordina (BPG-eskaintza), lurralde historiko, jarduera-adar (A-38), datu-mota eta neurri-motaren arabera* taularen datu guztiak kontsultatuko ditugu.



In [ ]:
# Taulako sarbide puntua (endpoint)
url = "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_170112_cpib_pib01d.px"

# Eskaeraren edukia: filtrorik gabe (taula osoa). Emaitza JSON-stat v.1.2 formatuan lortzen da, besterik esaten ez bada
query = { "query": []}


sesioa = requests.Session()
erantzuna = sesioa.post(url, json=query)                    # POST eskaera bat bidaltzen da taulako URLra (API kontsulta)
erantzuna_json = json.loads(erantzuna.content.decode("utf-8-sig"))     # Erantzuna testu irakurgarri bihurtzen du

emaitza = pyjstat.from_json_stat(erantzuna_json, naming="label", value="value")  # JSON-stat erantzuna etiketa deskribatzaileekin eskatzen dugu (label)
BPG = emaitza[0]                    # DataFramearen lehen zerrenda bakarrik interesatzen zaigu
BPG .head()

,lurralde historikoa,sektorea,datu-mota,neurri-mota,aldia,value
0,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1995,449513.0
1,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1996,439283.0
2,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1997,466735.0
3,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1998,564307.0
4,Euskal AE,"1. Nekazaritza, abeltzaintza eta arrantza",Maila,Uneko prezioak (mila euro),1999,605399.0


## Taula bateko datuak eskatu, kodeekin

Baliteke datu horiek aldagaien kodeekin eskuratu nahi izatea, eta ez etiketa deskribatzaileekin edo literalekin. Kasu horretan, izenak kode gisa (naming = "id") jeisteko eskatuko dugu, eta ez testu gisa (naming = "label").

In [ ]:
emaitza = pyjstat.from_json_stat(erantzuna_json, naming="id", value="value")  # JSON-stat erantzuna kodeekin eskatzen dugu (id)
BPG_kodeekin = emaitza[0]                    # DataFramearen lehen zerrenda bakarrik interesatzen zaigu
BPG_kodeekin.head()

,territorio histórico,sector,tipo de dato,tipo de medida,periodo,value
0,00,01,10,10,1995,449513.0
1,00,01,10,10,1996,439283.0
2,00,01,10,10,1997,466735.0
3,00,01,10,10,1998,564307.0
4,00,01,10,10,1999,605399.0


## Taula bateko datuen selekzio bat eskuratu

Datu gehiegi eskatzea, gero erdia bota behar izateko, espazioa eta baliabideak alferrik galtzea da. Horregatik, taularen datuen parte bat bakarrik interesatzen zaigunean, eta ez taularen eduki osoa, kontsulta bat egin behar dugu intereseko aldagaiak eta balioak hautatuta.

Kontsulta definitzeko, aldagaien izena/kodea eta balioak ezagutu behar dira. Aurreko puntuan ikusi dugu nola lortu taula baten metadatuak.

Baina urrats hori errazteko, Eustaten webguneak berak aldagaiak eta balioak hautatzeko interfaze bat du taula bakoitzarentzat. Interfaze horretan, erraz aukeratu ahal izango ditugu interesatzen zaizkigun datuak, eta kontsulta JSON formatuan lortu ahal izango dugu.

Adibidez, sexuaren araberako langabezia-tasaren datuak nahi izanez gero, datu
bankuko taula honetako aldagaiak hautatzen diren orrira joango ginateke:

https://www.eustat.eus/bankupx/pxweb/eu/DB/-/PX_050403_cpra_tab01.px

Eta interesatzen zaigun hautaketa egingo dugu: Euskal AEko langabezia-tasaren datuak, 2 sexuak (eta guztizkoa) eta azken 3 urteetako hiruhilekoko datuak hautatuko ditugu.

#### Hautaketaren adibidea:
![Langabezia aukeraketa](https://raw.githubusercontent.com/uxue-sudupe/API-Eustat/main/img/Langabezia_aukeraketa.png)


Aukeraketa egin ondoren, `Jarraitu` eta `Taula hau eskuragarri egin zure aplikazioan` sakatuko dugu. Atal honetan, eskaera egin behar zaion URLa eta json kontsulta agertuko dira. Kontsulta hori gure kodean sartuko dugu.


#### Kontsultaren adibidea:
![Langabezia kontsulta](https://raw.githubusercontent.com/uxue-sudupe/API-Eustat/main/img/Langabezia_kontsulta.png)



In [ ]:
url = "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_050403_cpra_tab01.px"
query = {"query": [{"code": "tasa (%)","selection": {"filter": "item", "values": ["30"]}},
        {"code": "territorio histórico","selection": {"filter": "item", "values": ["00"]}},
        {"code": "trimestre","selection": {"filter": "item","values": ["20", "30", "40","50"]}},
        {"code": "periodo","selection": {"filter": "item","values": ["2023", "2024","2025"]}}],
         "response": {"format": "json-stat"}}


sesioa = requests.Session()
erantzuna = sesioa.post(url, json=query)                 #  POST eskaera bat bidaltzen da taulako URLra (API kontsulta)
erantzuna_json = json.loads(erantzuna.content.decode('utf-8-sig'))    #  Erantzuna testu irakurgarri bihurtzen du
emaitzak = pyjstat.from_json_stat(erantzuna_json, naming='label', value='value')  #  JSON-stat erantzuna (datu estatistikoen formatu berezia) pandasen DataFramea bihurtzen du.
langabezia= emaitzak[0]        # DataFramearen lehen zerrenda bakarrik interesatzen zaigu
langabezia

,tasa (%),lurralde historikoa,sexua,hiruhilekoa,aldia,value
0,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2023,8.9
1,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2024,7.9
2,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2025,7.0
3,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2023,7.6
4,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2024,7.5
5,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2025,NaN
6,Langabezi tasa,Euskal AE,Guztira,3. Hiruhilekoa,2023,7.7
7,Langabezi tasa,Euskal AE,Guztira,3. Hiruhilekoa,2024,7.2
8,Langabezi tasa,Euskal AE,Guztira,3. Hiruhilekoa,2025,NaN
9,Langabezi tasa,Euskal AE,Guztira,4. Hiruhilekoa,2023,7.1


### Aldaketak kontsultan

`Filter`: `item` egiten denean aldagai baten gainean, filtratu nahi diren balioak hautatzen dira (bat edo gehiago).

Balio guztiak hautatzen badira, ez da aldagai horren gainean filtratzen, eta ez da kontsultan agertuko. Efektu bera du "lurralde historikoa" aldagaiaren kode-lerroarekin orain egingo dugun bezala `{"code":"variable"....}` kodearekin hasten den lerroa ezabatzeak ere.

`Top` filtroa ere erabil dezakegu, aldagaiaren azken balioak bakarrik hautatzeko. `Top` filtroa batez ere "aldia", "urtea" edo antzeko aldagaiekin erabiltzen da.



In [ ]:
url = "https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_050403_cpra_tab01.px"
query = {"query": [{"code": "tasa (%)","selection": {"filter": "item", "values": ["30"] }},
    #   {"code": "territorio histórico","selection": {"filter": "item", "values": ["00"]}},    lerro hau ezabatzen badugu, ez dugu "lurralde historikoa" aldagaia filtratzen
        {"code": "trimestre","selection": {"filter": "item","values": ["20", "30", "40","50"]}},
        {"code": "periodo","selection": {"filter": "top","values": ["2"]}}],             #   "top" filtroarekin, "periodo" aldagaiaren azken 2 balioak bakarrik eskatu ditugu
         "response": {"format": "json-stat"}}

sesioa = requests.Session()
erantzuna = sesioa.post(url, json=query)
erantzuna_json = json.loads(erantzuna.content.decode('utf-8-sig'))
emaitzak = pyjstat.from_json_stat(erantzuna_json, naming='label', value='value')
langabezia= emaitzak[0]
langabezia

,tasa (%),lurralde historikoa,sexua,hiruhilekoa,aldia,value
0,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2024,7.9
1,Langabezi tasa,Euskal AE,Guztira,1. Hiruhilekoa,2025,7.0
2,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2024,7.5
3,Langabezi tasa,Euskal AE,Guztira,2. Hiruhilekoa,2025,NaN
4,Langabezi tasa,Euskal AE,Guztira,3. Hiruhilekoa,2024,7.2
...,...,...,...,...,...,...
91,Langabezi tasa,Gipuzkoa,Emakumeak,2. Hiruhilekoa,2025,NaN
92,Langabezi tasa,Gipuzkoa,Emakumeak,3. Hiruhilekoa,2024,5.2
93,Langabezi tasa,Gipuzkoa,Emakumeak,3. Hiruhilekoa,2025,NaN
94,Langabezi tasa,Gipuzkoa,Emakumeak,4. Hiruhilekoa,2024,6.0


## Irteerako beste formatu batzuk

Tauletako datuetarako irteera lehenetsia JSON-stat da (1.2 bertsioa). JSON formatua da ohikoena APIetan, datuen tratamendu malgua ahalbidetzen duelako, script-etan (R, Python, etab.) automatikoki prozesatzeko egokia delako, eta
CSV bezalako artxibo lauetan aurkez daitezkeen anbiguotasunak saihesten dituelako.

APIari eskatutako datuak CSV eta excel formatuetan ere deskarga daitezke. Irteera horiek egokiagoak izan daitezke erabiltzaile ez hain teknikoentzat, sarrera gisa csv edo excel behar diren prozesuentzat (csv/excel bidez dashboard-ak sortzeko adibidez) edo helburua analisia ez denean, baizik eta APIak ematen dituen datuen deskarga erraztea publikoari.

Jarraian csv (3 mota) eta excel irteerak nola eskatu eta zein itxura duten azaltzen da. Irteera bakoitzaren kopia bat gordetzen da.



In [ ]:
# Irteera CSV - testu laua

url ="https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010901_cecv_ni11_1.px"
query = {"query": [],
  "response": {"format": "csv"}}


erantzuna_csv = requests.post(url, json=query)

#Erantzunaren edukia ikusteko. Urrats hau ez da beharrezkoa
testua_csv = erantzuna_csv.content.decode("windows-1252")
print(testua_csv[:1000])   #  JSONaren lehen 1000 karaktereen itxura ikusteko

# CSV fitxategia gorde
with open("ama_hizkuntza.csv", "wb") as f:
    f.write(erantzuna_csv.content)

"ama-hizkuntza","eskualdea","1989","1994","1999","2004","2009","2014","2019","2024"
"Guztira","Euskal AE",1997.2,1995.5,1991.0,1993.0,2051.6,2023.6,2079.0,2120.7
"Guztira","Arabako Lautada, Arabako Kantaurialdearekin",236.5,237.6,246.3,254.5,272.2,270.0,279.4,289.6
"Guztira","Arabako besteak",22.1,21.1,22.7,24.5,26.8,28.4,27.6,28.8
"Guztira","Gran Bilbao",871.6,862.7,847.8,839.2,842.5,804.4,813.0,833.1
"Guztira","Duranguesado",84.3,86.1,85.8,78.9,87.1,94.3,95.0,94.3
"Guztira","Gernika-Bermeo, Plentzia-Mungia eta Markina-Ondarroarekin",99.5,101.6,103.6,111.8,116.8,115.3,125.3,129.8
"Guztira","Arratia-Nerbioi, Enkartazioekin",47.5,48.1,47.8,49.3,50.2,53.3,53.5,54.0
"Guztira","Donostialdea, Bidasoa Beherarekin",353.5,357.5,361.9,364.6,374.6,375.6,389.6,391.5
"Guztira","Deba Garaia, Deba Beherea eta Urola-Kostaldearekin",174.7,173.8,172.4,171.0,176.4,174.2,182.7,186.2
"Guztira","Tolosaldea, Goierrirekin",107.5,106.8,102.8,99.3,105.0,108.1,112.9,113.2
"Euskara","Euskal AE",399.3,


In [ ]:
#  CSV2 moduko irteera - "pivot-friendly" formatua, taula dinamikoentzat egokia

url ="https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010901_cecv_ni11_1.px"
query = {"query": [],
  "response": {"format": "csv2"}}


erantzuna_csv2 = requests.post(url, json=query)

# CSV fitxategia gorde
with open("ama_hizkuntza_csv2.csv", "wb") as f:
    f.write(erantzuna_csv2.content)

In [ ]:
# CSV3 moduko irteera - CSV2 bezala, baina kodeekin testua izan beharrean

url ="https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010901_cecv_ni11_1.px"
query = {"query": [],
  "response": {"format": "csv3"}}


erantzuna_csv3 = requests.post(url, json=query)

# CSV fitxategia gorde
with open("ama_hizkuntza_csv3.csv", "wb") as f:
    f.write(erantzuna_csv3.content)


In [ ]:
# Excel irteera

url ="https://www.eustat.eus/bankupx/api/v1/eu/DB/PX_010901_cecv_ni11_1.px"
query = {"query": [],
  "response": {"format": "xlsx"}}

erantzuna = requests.post(url, json=query)

# Excel fitxategia gorde
with open("ama_hizkuntza.xlsx", "wb") as f:
    f.write(erantzuna.content)